# Week7 - Decision Tree Lab

* Train-test split
* Train a decison tree model
* Train a random forest model
* Evaluate the models
* Explain findings

In [3]:
#print df
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/log_reg/employee-turnover-balanced.csv')
df

,left_company,age,frequency_of_travel,department,commuting_distance,education,satisfaction_with_environment,gender,seniority_level,position,satisfaction_with_job,married_or_single,last_raise_pct,last_performance_rating,total_years_working,years_at_company,years_in_current_job,years_since_last_promotion,years_with_current_supervisor
0,No,37,Travel_Rarely,Sales,16,4,4,Male,2,Sales Executive,3,Divorced,19,3,9,1,0,0,0
1,No,39,Travel_Rarely,Research & Development,3,2,3,Male,2,Laboratory Technician,3,Divorced,15,3,11,10,8,0,7
2,No,52,Travel_Frequently,Research & Development,25,4,3,Female,4,Manufacturing Director,4,Married,22,4,31,9,8,0,0
3,No,50,Non-Travel,Sales,1,3,4,Female,2,Sales Executive,3,Married,12,3,19,18,7,0,13
4,No,44,Travel_Rarely,Research & Development,4,3,4,Male,2,Healthcare Representative,2,Single,12,3,10,5,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Yes,39,Travel_Rarely,Sales,3,2,4,Female,2,Sales Executive,3,Married,18,3,12,1,0,0,0
996,Yes,26,Travel_Rarely,Sales,4,4,4,Male,2,Sales Executive,4,Single,12,3,8,8,7,7,4
997,Yes,18,Travel_Frequently,Sales,5,3,2,Male,1,Sales Representative,2,Single,14,3,0,0,0,0,0
998,Yes,28,Travel_Rarely,Research & Development,2,4,1,Male,1,Research Scientist,4,Married,13,3,5,3,2,2,2


In [13]:
#info of df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   left_company                   1000 non-null   object
 1   age                            1000 non-null   int64 
 2   frequency_of_travel            1000 non-null   object
 3   department                     1000 non-null   object
 4   commuting_distance             1000 non-null   int64 
 5   education                      1000 non-null   int64 
 6   satisfaction_with_environment  1000 non-null   int64 
 7   gender                         1000 non-null   object
 8   seniority_level                1000 non-null   int64 
 9   position                       1000 non-null   object
 10  satisfaction_with_job          1000 non-null   int64 
 11  married_or_single              1000 non-null   object
 12  last_raise_pct                 1000 non-null   int64 
 13  last

In [16]:
# numerical_vars = []
df_numerical_vars = df.select_dtypes(include='int64')

# categorical_vars = []
df_categorical_vars = df.select_dtypes(include='object')

In [17]:
# insert code here
#splitted the data into train and test
from sklearn.model_selection import train_test_split

y = df['left_company']
X = df.iloc[:, 1:]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=124)

print(f'Training samples: {X_train.shape[0]:,}')
print(f'Test samples: {X_test.shape[0]:,}')

Training samples: 800
Test samples: 200


In [33]:
# insert code here

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


polys = []
non_polys = df_numerical_vars.columns
ohes = df_categorical_vars.columns.drop('left_company')

poly_pipeline = Pipeline([
    ('add_polynomials', PolynomialFeatures(2, include_bias=False)),
    ('standardize_poly', StandardScaler())]
)
    
processing_pipeline = ColumnTransformer(transformers=[
    ('poly_processing', poly_pipeline, polys),
    ('nonpoly_scaling', StandardScaler(), non_polys),
    ('dummys', OneHotEncoder(drop='first'), ohes)]
)

In [35]:
# insert code here
import warnings
warnings.filterwarnings('ignore')

modeling_pipeline = Pipeline([('data_processing', processing_pipeline), 
                              ('logreg', LogisticRegression(penalty=None, max_iter=500))
                              ])

param_grid = [
  {'logreg__class_weight': [None, 'balanced'], 'logreg__C':[0.01, 0.1, 1, 10, 100]}
 ]

gcv_results = GridSearchCV(estimator=modeling_pipeline, 
                           param_grid=param_grid, scoring='recall',refit=True)
gcv_results = gcv_results.fit(X_train, y_train)

gcv_results.best_estimator_

Pipeline(steps=[('data_processing',
                 ColumnTransformer(transformers=[('poly_processing',
                                                  Pipeline(steps=[('add_polynomials',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('standardize_poly',
                                                                   StandardScaler())]),
                                                  []),
                                                 ('nonpoly_scaling',
                                                  StandardScaler(),
                                                  Index(['age', 'commuting_distance', 'education',
       'satisfaction_with_environment', 'seniority_level',
       'satisfaction...,
       'total_years_working', 'years_at_company', 'years_in_current_job',
       'years_since_last_promotion', 'years_with_current_supervisor'],
      dtype='object')),
                                                 ('dummys',
                                                  OneHotEncoder(drop='first'),
                                                  Index(['frequency_of_travel', 'department', 'gender', 'position',
       'married_or_single'],
      dtype='object'))])),
                ('logreg',
                 LogisticRegression(C=0.01, max_iter=500, penalty=None))])

In [36]:
modeling_pipeline.fit(X_train,y_train)
print("Training dataset score is" ,modeling_pipeline.score(X_train, y_train))
print("Testing dataset score is" ,modeling_pipeline.score(X_test, y_test))

Training dataset score is 0.7275
Testing dataset score is 0.67


In [37]:
y_testp = gcv_results.predict(X_test)
y_testpr_lr = gcv_results.predict_proba(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_testp))

              precision    recall  f1-score   support

          No       0.65      0.62      0.64        93
         Yes       0.68      0.71      0.70       107

    accuracy                           0.67       200
   macro avg       0.67      0.67      0.67       200
weighted avg       0.67      0.67      0.67       200



In [40]:
# insert code here

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV


polys = []
non_polys = df_numerical_vars.columns
ohes = df_categorical_vars.columns.drop('left_company')

poly_pipeline = Pipeline([
    ('add_polynomials', PolynomialFeatures(2, include_bias=False)),
    ('standardize_poly', StandardScaler())]
)
    
processing_pipeline = ColumnTransformer(transformers=[
    ('poly_processing', poly_pipeline, polys),
    ('nonpoly_scaling', StandardScaler(), non_polys),
    ('dummys', OneHotEncoder(drop='first'), ohes)]
)

modeling_pipeline = Pipeline([
    ('data_processing', processing_pipeline),
    ('dt', DecisionTreeClassifier())]
)

param_grid = [
  {'dt__max_depth': [2, 5, 10, 15, 20],
   'dt__min_samples_split':[3, 5, 10, 20, 40],
   'dt__min_samples_leaf': [2, 5],
   'dt__class_weight':[None, 'balanced']
  }
 ]

gcv_results = GridSearchCV(estimator=modeling_pipeline, 
                           param_grid=param_grid, scoring='recall',refit=True)
gcv_results = gcv_results.fit(X_train, y_train)

gcv_results.best_estimator_

Pipeline(steps=[('data_processing',
                 ColumnTransformer(transformers=[('poly_processing',
                                                  Pipeline(steps=[('add_polynomials',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('standardize_poly',
                                                                   StandardScaler())]),
                                                  []),
                                                 ('nonpoly_scaling',
                                                  StandardScaler(),
                                                  Index(['age', 'commuting_distance', 'education',
       'satisfaction_with_environment', 'seniority_level',
       'satisfaction...
       'total_years_working', 'years_at_company', 'years_in_current_job',
       'years_since_last_promotion', 'years_with_current_supervisor'],
      dtype='object')),
                                                 ('dummys',
                                                  OneHotEncoder(drop='first'),
                                                  Index(['frequency_of_travel', 'department', 'gender', 'position',
       'married_or_single'],
      dtype='object'))])),
                ('dt',
                 DecisionTreeClassifier(max_depth=2, min_samples_leaf=2,
                                        min_samples_split=3))])

In [41]:
y_testp = gcv_results.predict(X_test)
y_testp_dt = gcv_results.predict_proba(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_testp))

              precision    recall  f1-score   support

          No       0.62      0.53      0.57        93
         Yes       0.64      0.72      0.68       107

    accuracy                           0.63       200
   macro avg       0.63      0.62      0.62       200
weighted avg       0.63      0.63      0.63       200



In [44]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

polys = []
non_polys = df_numerical_vars.columns
ohes = df_categorical_vars.columns.drop('left_company')

poly_pipeline = Pipeline([
    ('add_polynomials', PolynomialFeatures(2, include_bias=False)),
    ('standardize_poly', StandardScaler())]
)
    
processing_pipeline = ColumnTransformer(transformers=[
    ('poly_processing', poly_pipeline, polys),
    ('nonpoly_scaling', StandardScaler(), non_polys),
    ('dummys', OneHotEncoder(drop='first'), ohes)]
)

rf_pipeline = Pipeline([
    ('data_processing', processing_pipeline),
    ('rf', RandomForestClassifier())]
)

param_grid = [{'rf__max_depth': [5, 8, 10, 12],
               'rf__n_estimators': [10, 50, 100],
               'rf__class_weight': [None, 'balanced', 'balanced_subsample'],
               'rf__max_samples': [200,400,600]
              }]

rf_results = GridSearchCV(estimator=rf_pipeline, param_grid=param_grid, scoring='recall', refit=True)
rf_results = rf_results.fit(X_train, y_train)
rf_yhat = rf_results.predict(X_test)

rf_results.best_estimator_

Pipeline(steps=[('data_processing',
                 ColumnTransformer(transformers=[('poly_processing',
                                                  Pipeline(steps=[('add_polynomials',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('standardize_poly',
                                                                   StandardScaler())]),
                                                  []),
                                                 ('nonpoly_scaling',
                                                  StandardScaler(),
                                                  Index(['age', 'commuting_distance', 'education',
       'satisfaction_with_environment', 'seniority_level',
       'satisfaction...
       'total_years_working', 'years_at_company', 'years_in_current_job',
       'years_since_last_promotion', 'years_with_current_supervisor'],
      dtype='object')),
                                                 ('dummys',
                                                  OneHotEncoder(drop='first'),
                                                  Index(['frequency_of_travel', 'department', 'gender', 'position',
       'married_or_single'],
      dtype='object'))])),
                ('rf',
                 RandomForestClassifier(max_depth=5, max_samples=200,
                                        n_estimators=10))])

In [45]:
y_testp = rf_results.predict(X_test)
y_testp_rf = rf_results.predict_proba(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_testp))

              precision    recall  f1-score   support

          No       0.69      0.71      0.70        93
         Yes       0.74      0.72      0.73       107

    accuracy                           0.71       200
   macro avg       0.71      0.71      0.71       200
weighted avg       0.72      0.71      0.72       200



# findings

# Results:
    Accuracy (logistic regression)= 0.67
    Accuracy (Decision tree) = 0.63
    Accuracy (Random Forest) = 0.71
    
    we can conclude that the Random forest classifier has increased the accuracy and performed better on test data.